<a href="https://colab.research.google.com/github/YassineZITI/IT-or-Not-Books-/blob/main/itbookstore_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction



<h2>IT or Not Classification</h2>
<p>In a previous project i built a website where you can look for data science books, and in that website you can upload books.<p/>
<p>So to help me verify that the user will upload only it books i have to build this classifier.</p>
<p>So in this series of notebooks we will try to build this classifier using NLP .</p>




# Data Collection

we will use itbook.store website and its free api to collect the data

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#i will only scrape the books related to Data Science 
categories = ['Database','TensorFlow','Neural Networks','Hadoop',
              'NumPy','Visualization','Machine Learning',
              'Analytics','Apache','Big Data','Scala','Microsoft']

In [ ]:
len(categories)

12

In [ ]:
#get books from each category  
def get_total(category):
  url = 'https://api.itbook.store/1.0/search/{}'.format(category)
  total = int(requests.get(url).json()['total'])
  return total
def get_data_category(category,page):
  data = []
  url = 'https://api.itbook.store/1.0/search/{}/{}'.format(category,page)
  data.extend(requests.get(url).json()['books'])
  for i in range(len(data)):
    data[i]['category'] = category
  return data
def get_complete_data_category(category):
  data = []
  pages = min(120,get_total(category)//10)
  for page in range(pages):
      data.extend(get_data_category(category,page))
  for i in range(len(data)):
    url = data[i]['url']
    response = requests.get(url)
    meta = requests.get('https://api.itbook.store/1.0/books/'+ data[i]['isbn13']).json()
    #author
    data[i]['authors'] = meta['authors']
    #publisher
    data[i]['publisher'] = meta['publisher']
    #published
    data[i]['year'] = meta['year']
    #description
    soup = BeautifulSoup(response.content,'html.parser')
    desc = soup.find('div',itemprop='description').text
    desc = re.sub('\n|\r',' ',desc)
    data[i]['description'] = desc
    #download link
    try:
      down_link = soup.find('a' ,title='Free Download')['href']
    except:
      down_link = ''
    data[i]['downlink'] = down_link
    #buy link
    buy_link = 'https://itbook.store/go/buy/' + data[i]['isbn13']
    data[i]['buylink'] = buy_link
    #pages
    data[i]['pages'] = meta['pages']
    #rating
    data[i]['rating'] = int(meta['rating'])
  return data
  

def data_collection():
  data = []
  for category in categories :
    data.extend(get_complete_data_category(category))
  
  
  return data

In [ ]:
data = data_collection()

In [ ]:
len(data)

5830

In [ ]:
#create a data frame
import pandas as pd
df = pd.DataFrame(data)

In [ ]:
data = df.drop_duplicates(subset=['isbn13']).to_dict('records')

In [ ]:
len(data)

3411

In [ ]:
counts = {c:0 for c in categories}
for book in data:
  counts[book['category']]+=1
counts

{'Analytics': 90,
 'Apache': 66,
 'Big Data': 314,
 'Database': 790,
 'Hadoop': 56,
 'Machine Learning': 686,
 'Microsoft': 597,
 'Neural Networks': 268,
 'NumPy': 23,
 'Scala': 260,
 'TensorFlow': 70,
 'Visualization': 191}

# Save Data


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = df[['title','description']]

In [ ]:
df.loc[:,'category'] = 'IT'

In [ ]:
df.head()

,title,description,category
0,"Expert Oracle Database Architecture, 2nd Edition","Now in its second edition, this best-selling ...",IT
1,Expert Indexing in Oracle Database 11g,Expert Indexing in Oracle Database 11g is abo...,IT
2,Databases for Small Business,This book covers the practical aspects of dat...,IT
3,"Next Generation Databases: NoSQL, NewSQL, and ...","This is a book for enterprise architects, dat...",IT
4,"Practical PHP 7, MySQL 8, and MariaDB Website ...","Build interactive, database-driven websites w...",IT


In [ ]:
import pickle
with open('/content/drive/My Drive/IT.txt', 'wb') as f:
    pickle.dump( df,f)